# Day 12: Hill Climbing Algorithm

[https://adventofcode.com/2022/day/12](https://adventofcode.com/2022/day/12)

## Description

### Part One

You try contacting the Elves using your <span title="When you look up the specs for your handheld device, every field just says &quot;plot&quot;.">handheld device</span>, but the river you're following must be too low to get a decent signal.

You ask the device for a heightmap of the surrounding area (your puzzle input). The heightmap shows the local area from above broken into a grid; the elevation of each square of the grid is given by a single lowercase letter, where `a` is the lowest elevation, `b` is the next-lowest, and so on up to the highest elevation, `z`.

Also included on the heightmap are marks for your current position (`S`) and the location that should get the best signal (`E`). Your current position (`S`) has elevation `a`, and the location that should get the best signal (`E`) has elevation `z`.

You'd like to reach `E`, but to save energy, you should do it in _as few steps as possible_. During each step, you can move exactly one square up, down, left, or right. To avoid needing to get out your climbing gear, the elevation of the destination square can be _at most one higher_ than the elevation of your current square; that is, if your current elevation is `m`, you could step to elevation `n`, but not to elevation `o`. (This also means that the elevation of the destination square can be much lower than the elevation of your current square.)

For example:

    Sabqponm
    abcryxxl
    accszExk
    acctuvwj
    abdefghi
    

Here, you start in the top-left corner; your goal is near the middle. You could start by moving down or right, but eventually you'll need to head toward the `e` at the bottom. From there, you can spiral around to the goal:

    v..v<<<<
    >v.vv<<^
    .>vv>E^^
    ..v>>>^^
    ..>>>>>^
    

In the above diagram, the symbols indicate whether the path exits each square moving up (`^`), down (`v`), left (`<`), or right (`>`). The location that should get the best signal is still `E`, and `.` marks unvisited squares.

This path reaches the goal in _`31`_ steps, the fewest possible.

_What is the fewest steps required to move from your current position to the location that should get the best signal?_


In [3]:
from collections import deque
from pathlib import Path

TEST = """Sabqponm
abcryxxl
accszExk
acctuvwj
abdefghi""".splitlines()

EXPECTED_1 = 31
EXPECTED_2 = 29

DATA = Path("input/data12.txt").read_text().splitlines()


def parse(data: list[str]) -> tuple[list[list[int]], complex, complex, int, int]:
    rows = [
        [0 if c == "S" else 25 if c == "E" else ord(c) - ord("a") for c in row]
        for row in data
    ]
    for y, row in enumerate(data):
        if "S" in row:
            start = row.find("S") + y * 1j
        if "E" in row:
            end = row.find("E") + y * 1j
    # print(f"start {start}, end {end}")

    return rows, start, end, len(row), len(data)


def neighbours(pos: complex, width: int, height: int):
    if pos.real >= 0:
        yield pos - 1
    if pos.real < width - 1:
        yield pos + 1
    if pos.imag >= 0:
        yield pos - 1j
    if pos.imag < height - 1:
        yield pos + 1j


def score_1(data: list[str]) -> int:
    grid, start, end, width, height = parse(data)
    steps = {start: 0}
    queue = deque([start])
    while end not in steps and queue:
        pos = queue.popleft()
        h = grid[int(pos.imag)][int(pos.real)]
        count = steps[pos] + 1
        for nxt in neighbours(pos, width, height):
            nh = grid[int(nxt.imag)][int(nxt.real)]
            if nxt not in steps and nh - h <= 1:
                steps[nxt] = count
                queue.append(nxt)

    return steps[end]


test_score = score_1(TEST)
print("test", test_score)
assert test_score == EXPECTED_1
print("part 1", score_1(DATA))


test 31
part 1 437


<div class="alert alert-info">I used complex numbers for the coordinates because lots of other people seem to do that. Not sure it gained anything. Nice chance to use a `deque`, I don't often get to use them.</div>


### Part Two

As you walk up the hill, you suspect that the Elves will want to turn this into a hiking trail. The beginning isn't very scenic, though; perhaps you can find a better starting point.

To maximize exercise while hiking, the trail should start as low as possible: elevation `a`. The goal is still the square marked `E`. However, the trail should still be direct, taking the fewest steps to reach its goal. So, you'll need to find the shortest path from _any square at elevation `a`_ to the square marked `E`.

Again consider the example from above:

    Sabqponm
    abcryxxl
    accszExk
    acctuvwj
    abdefghi
    

Now, there are six choices for starting position (five marked `a`, plus the square marked `S` that counts as being at elevation `a`). If you start at the bottom-left square, you can reach the goal most quickly:

    ...v<<<<
    ...vv<<^
    ...v>E^^
    .>v>>>^^
    >^>>>>>^
    

This path reaches the goal in only _`29`_ steps, the fewest possible.

_What is the fewest steps required to move starting from any square with elevation `a` to the location that should get the best signal?_


In [4]:

def score_2(data: list[str]) -> int:
    grid, start, end, width, height = parse(data)
    steps = {end: 0}
    queue = deque([end])
    while queue:
        pos = queue.popleft()
        h = grid[int(pos.imag)][int(pos.real)]

        count = steps[pos] + 1
        for nxt in neighbours(pos, width, height):
            nh = grid[int(nxt.imag)][int(nxt.real)]

            if nxt not in steps and h - nh <= 1:
                if nh == 0:
                    return count
                steps[nxt] = count
                queue.append(nxt)

    return steps[start]


if EXPECTED_2 is not None:
    test_score = score_2(TEST)
    print("test", test_score)
    assert test_score == EXPECTED_2
    print("part 2", score_2(DATA))


test 29
part 2 430


<div class="alert alert-info">Choice here was to run the algorithm backwards (which I did) or just run it forward starting from every possible 0 height location and choose the smallest answer.</div>